##  Checking english tweets

Some of this code calls functions from the "process_gdrive_files" notebooks. I've extracted the code here though so that it is more salient

In [ ]:
import fasttext
model = fasttext.load_model("../../../data/lid.176.bin")

def detect_en(x):
    try:
        return model.predict(x)[0][0]=="__label__en"
    except:
        return False

In [2]:

# Sample 100 lines from each csv in a folder and come up with a score
# reflecting the likelihood this is an english speaking account
def process(folder,sample_size=100):
    fileslist:list = get_files(folder)
    print(f"Retrieved {len(fileslist)} files")
    results = {}
    for f in fileslist:
        result = []
        if not re.match("[^.]+\.csv",f['title']):
            print(f"Skipping {f['title']}")
            continue
        f.GetContentFile(f['title'])
        data = pd.read_csv(f['title'],nrows = sample_size)
        if ("text" not in data.columns):
            print(f"{f['title']} is missing data")
            results[f['title']] = 0
        else:
            x = data.text.tolist()
            for line in x:
                line = re.sub("\n+"," ",line)
                result.append(detect_en(line))
            
            results[f['title']] = sum(result)/100
        os.remove(f['title'])
    return results